<a href="https://colab.research.google.com/github/gargarchit/Federated-Learning-on-MNIST/blob/master/Federated_Learning_on_MNIST_using_PyTorch_%2B_PySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Need to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form**

In [0]:
import torch
from torchvision import datasets, transforms

<h2>Installing and Importing PySyft

In [0]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

In [0]:
import syft as sy
hook = sy.TorchHook(torch)

# Create a couple of workers
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
#We define the setting of the learning task

class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

torch.manual_seed(args.seed)

In [0]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


<h2>Data loading and sending to workers</h2>

In [0]:
# FederatedDataLoader 
# we distribute the dataset across all the workers, it's now a FederatedDataset
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]) 
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform).federate((bob, alice))
mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=transform)

federated_train_loader = sy.FederatedDataLoader(mnist_trainset, batch_size=args.batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=args.test_batch_size, shuffle=True)

<h2>Network Architecture</h2>

In [0]:
from torch import nn, optim
import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [0]:
class Classifier(nn.Module):
    """
    Forward Convolutional Neural Network Architecture model
    
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
model = Classifier()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # iterate through each worker's dataset
        
        model.send(data.location) #send the model to the right location
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        optimizer.step()  # 5) change those weights
        model.get()  # get the model back (with gradients)
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() #get the loss back
            print('Epoch: {} [Training: {:.0f}%]\tLoss: {:.6f}'.format(epoch, 100. * batch_idx / len(federated_train_loader), loss.item()))
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Epoch: 1 [Training: 0%]	Loss: 2.302580
Epoch: 1 [Training: 3%]	Loss: 2.173574
Epoch: 1 [Training: 6%]	Loss: 1.956030
Epoch: 1 [Training: 10%]	Loss: 1.320343
Epoch: 1 [Training: 13%]	Loss: 0.835926
Epoch: 1 [Training: 16%]	Loss: 0.661665
Epoch: 1 [Training: 19%]	Loss: 0.719910
Epoch: 1 [Training: 22%]	Loss: 0.372965
Epoch: 1 [Training: 26%]	Loss: 0.568405
Epoch: 1 [Training: 29%]	Loss: 0.376864
Epoch: 1 [Training: 32%]	Loss: 0.306380
Epoch: 1 [Training: 35%]	Loss: 0.180607
Epoch: 1 [Training: 38%]	Loss: 0.434034
Epoch: 1 [Training: 42%]	Loss: 0.366301
Epoch: 1 [Training: 45%]	Loss: 0.284107
Epoch: 1 [Training: 48%]	Loss: 0.230000
Epoch: 1 [Training: 51%]	Loss: 0.243508
Epoch: 1 [Training: 54%]	Loss: 0.344105
Epoch: 1 [Training: 58%]	Loss: 0.269959
Epoch: 1 [Training: 61%]	Loss: 0.132271
Epoch: 1 [Training: 64%]	Loss: 0.138609
Epoch: 1 [Training: 67%]	Loss: 0.174517
Epoch: 1 [Training: 70%]	Loss: 0.082769
Epoch: 1 [Training: 74%]	Loss: 0.164831
Epoch: 1 [Training: 77%]	Loss: 0.207887
Epo

In [0]:
print("Accuracy Obtained {:.4f}%".format( 100. * correct / len(test_loader.dataset)))

Accuracy Obtained 98.9000%


<h3> Do Check out [The Future of AI: Federated Learning](https://medium.com/secure-and-private-ai-writing-challenge/the-future-of-ai-federated-learning-fe977cdd3c62)